<a href="https://colab.research.google.com/github/tariqulq-cyber/Pemrosesan-teks-praktek/blob/main/preprocesing/pemrosesan_teks_teori_preprocesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nltk Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.3 MB/s eta 0:00:00


In [10]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt_tab


      Unzipping tokenizers/punkt_tab.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pandas as pd
import os
import sys
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# =======================================================
# 1. KONFIGURASI AWAL & SETUP LIBRARY
# =======================================================

NAMA_FILE_INPUT = '/content/drive/MyDrive/pemrosesan teks teori/casefolding_growtopia.csv' # File Input Anda
NAMA_FILE_OUTPUT = 'preprocesing_growtopia.csv' # Nama file CSV output
KOLOM_TEKS = 'content' # Nama kolom yang berisi teks

# --- SETUP NLP ---
# 1. Stopword Removal (Menggunakan list standar NLTK)
list_stopwords = set(stopwords.words('indonesian'))
# Tambahkan stopword informal yang sering muncul di komentar (PENTING)
list_stopwords_tambahan = {
    # Kata Ganti/Keterangan/Partikel Informal
    'gua','itu','ini','yg','aja', 'sih', 'dong', 'deh', 'nya', 'nih', 'tuh', 'loh', 'kek',
    'pake', 'msh', 'udh', 'emg', 'trs', 'dlm', 'sm', 'tp', 'bgt', 'gw','ni','apa','ya','yah','dh','dah','woi',

    # Sapaan/Suku Kata/Singkatan
    'cuy', 'min', 'admin', 'gan', 'sis', 'kak', 'kalo', 'klo', 'gt',
    'gitu', 'sama', 'ama', 'bisa', 'kalian', 'mereka', 'sii', 'ah', 'eh',
    'hmm', 'wkwk', 'hehe', 'hihi','sbg','dgn',

    # Kata Kerja/Hubung Umum yang sering diulang
    'adalah', 'iya', 'sudah', 'telah', 'mau', 'ingin', 'dan', 'di',
    'growtopia', 'gsc' # Contoh kata spesifik dari dataset
}
list_stopwords = list_stopwords.union(list_stopwords_tambahan)
print(f"[INFO] Stopword berhasil digabungkan (UNION). Total: {len(list_stopwords)} kata.")
# 2. Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# --- KAMUS NORMALISASI KATA (Contoh) ---
normalisasi_map = {
    'gaje': 'tidak jelas',
    'bgt': 'banget',
    'bikin': 'membuat',
    'ga': 'tidak',
    'gk': 'tidak',
    'gak': 'tidak',
    'udh': 'sudah',
    'mabar': 'main bareng',
    'krn': 'karena',
    'cpt': 'cepat',
    'kzl': 'kesal',
    'tp': 'tapi',
    'anj': 'anjing',
    'log': 'login',
    'gem':'game'



}

# --- Fungsi Normalisasi Kata (SAMA) ---
def normalisasi_kata(teks, mapping):
    if pd.isna(teks): return ""
    list_kata = teks.split()
    kata_normal = [mapping.get(kata, kata) for kata in list_kata]
    return " ".join(kata_normal)

print("=======================================================")
print(f"Memulai Preprocessing NLP (Stopword Intensif) untuk file: {NAMA_FILE_INPUT}")
print("=======================================================")

# =======================================================
# 2 & 3. MUAT DATA DAN HAPUS DUPLIKAT (SAMA)
# =======================================================
try:
    df = pd.read_csv(NAMA_FILE_INPUT)
    if KOLOM_TEKS not in df.columns:
        print(f"[ERROR] Kolom '{KOLOM_TEKS}' tidak ditemukan.")
        sys.exit(1)

    df[KOLOM_TEKS] = df[KOLOM_TEKS].fillna('').astype(str).str.lower()
    total_awal = len(df)
    df.drop_duplicates(subset=[KOLOM_TEKS], inplace=True)
    print(f"[INFO] Data dimuat. Duplikat terhapus: {total_awal - len(df)} baris.")

except Exception as e:
    print(f"[ERROR] Gagal memuat file CSV: {e}")
    sys.exit(1)

# =======================================================
# 4. RANTAI PREPROCESSING LENGKAP (MODIFIKASI STOPWORD)
# =======================================================

def preprocessing_lengkap(teks):
    # 1. Normalisasi Kata (Slang/Tidak Baku)
    teks = normalisasi_kata(teks, normalisasi_map)

    # 2. Penghapusan Tanda Baca, Simbol, dan Angka (Cleaning)
    teks = re.sub(r'[^a-zA-Z\s]', '', teks)

    # 3. Tokenisasi
    tokens = word_tokenize(teks)

    # 4. >>> STOPWORD REMOVAL STANDAR (HANYA MENGHILANGKAN SEMUA KATA FUNGSI) <<<
    # Perhatikan: Tidak ada filter panjang kata <= 3 di sini
    tokens_no_stop = [
        word for word in tokens
        if word not in list_stopwords and len(word) > 1 # Filter kata 1 huruf (biasanya sisa cleaning)
    ]

    # 5. Stemming (Mengembalikan ke kata dasar)
    tokens_stemmed = [stemmer.stem(word) for word in tokens_no_stop]

    # Menggabungkan kembali menjadi string bersih
    return " ".join(tokens_stemmed)

print("\n[INFO] Mulai menerapkan rantai Preprocessing Lengkap (Stopword Intensif)...")
df['content_processed_FINAL'] = df[KOLOM_TEKS].apply(preprocessing_lengkap)
print("[INFO] Preprocessing Lengkap selesai.")


# =======================================================
# 5. EKSPOR DATA KE CSV BARU (SAMA)
# =======================================================

df_export = df.copy()

print("\n[PREVIEW] Perbandingan data MENTAH vs. data AKHIR (5 Baris):")
print(df_export[[KOLOM_TEKS, 'content_processed_FINAL']].head().to_string(index=False))

try:
    kolom_final = list(df.columns)
    with open(NAMA_FILE_OUTPUT, 'w', newline='', encoding='utf-8') as f:
        df_export.to_csv(f, index=False, encoding='utf-8', columns=kolom_final)

    print(f"\n[SUKSES EKSPOR] Data Preprocessing Lengkap disimpan ke file:")
    print(f"{os.path.abspath(NAMA_FILE_OUTPUT)}")
    print("=======================================================")

except Exception as e:
    print("=======================================================")
    print(f"\n[!!! KEGAGALAN KRITIS EKSPOR CSV !!!] Tipe Kesalahan: {type(e).__name__}, Detail: {e}")
    print("=======================================================")

[INFO] Stopword berhasil digabungkan (UNION). Total: 806 kata.
Memulai Preprocessing NLP (Stopword Intensif) untuk file: /content/drive/MyDrive/pemrosesan teks teori/casefolding_growtopia.csv
[INFO] Data dimuat. Duplikat terhapus: 50 baris.

[INFO] Mulai menerapkan rantai Preprocessing Lengkap (Stopword Intensif)...
[INFO] Preprocessing Lengkap selesai.

[PREVIEW] Perbandingan data MENTAH vs. data AKHIR (5 Baris):
                                                                                                                                                   content                                                                                                                         content_processed_FINAL
                                                                              game nya susah banget anj cuman buat login bikin akun di persulit game konfl                                                                                      game susah banget anjing cuman login akun s